# Pragmatic LLM Application Development: From RAG Pipleines to AI Agents
## Introduction

A practical guide that introduces two forms of LLM Applications: RAG (Retrieval-Augmented Generation) pipelines and AI Agents.

This guide is designed to take you on a journey that develops your understanding of LLM Applications, starting with implementations without abstraction frameworks, and later introducing the implementation of RAG pipelines, AI agents, and other LLM application components using frameworks and libraries that alleviate the implementation burden for AI Stack Engineers.



### Key topics covered:

1. **Document Model and MongoDB Integration**: Introduces the Document model and its integration with MongoDB within LLM applications.

2. **RAG Pipeline Fundamentals**: Guides you through the key processes within a RAG pipeline, including data embedding, data ingestion, and handling user queries.

3. **MongoDB Vector Database Integration**: Guides you through the development of a RAG pipeline connected to a MongoDB Vector Database and utilizing OpenAI's models.

4. **MongoDB Aggregation Pipelines**: Introduces MongoDB Aggregation pipelines and stages for efficient data retrieval implementation within pipelines.

5. **LLM Abstraction Frameworks**: Showcases the development of RAG pipelines using widely-used LLM abstraction frameworks such as LangChain, LlamaIndex, and HayStack.

6. **Data Handling in LLM Applications**: Presents methods for handling data in LLM applications using tools such as Pydantic and Pandas.

7. **AI Agent Implementation**: Introduces the implementation of AI Agents using libraries such as LangChain and LlamaIndex.

8. **LLM Application Optimization**: Introduces techniques for optimizing LLM Applications, such as prompt compression using the LLMLingua library.

# Table of Content

[**Part 1: Vanilla RAG Application**](#scrollTo=hlnz3AIYn5DK)
- [1.1 Synthetic Data Creation](#scrollTo=VXlm_J_TokJp)
- [1.2 Embedding Data for Vector Search](#scrollTo=0AOQw0Caosxu)
- [1.3 Data Ingestion into MongoDB Database](#scrollTo=MhO4jWndsWjR)
- [1.4 Vector Search Index Creation](#scrollTo=B8VZ-c4qt92b)
- [1.5 RAG with MongoDB](#scrollTo=EC6nU1NSuFqO)
- [1.6 Handling User Query](#scrollTo=4UaKjc5nugfd)
- [1.7 Handling User Query With Prompt Compression (LLMLingua)](#scrollTo=BKdB25EMukQO)

[**Part 2: RAG Application With Abstraction Frameworks**](#scrollTo=ALrfaObSteOs)
- [2.1 RAG with LangChain and MongoDB](#scrollTo=DWK6DxuQjmhp)
    - [2.1.3 Prompt Compression with LangChain and LLMLingua](#scrollTo=rnSuWk2cqxtq)
- 2.2 RAG with LlamaIndex and MongoDB
- 2.3 RAG with HayStack and MongoDB

[**Part 3: AI Agent Application: HR Use Case**]()

## Part 1: Vanilla RAG Application

## Installation

In [1]:
%pip install -q pandas openai pymongo llmlingua accelerate

Note: you may need to restart the kernel to use updated packages.


## Set Up OpenAI Keys

In [2]:
import os
import openai
from openai import OpenAI

openai_client = OpenAI()
OPEN_AI_MODEL = "gpt-4o"
OPEN_AI_EMBEDDING_MODEL = "text-embedding-3-small"
OPEN_AI_EMBEDDING_MODEL_DIMENSION = 1536

## 1.1 Synthetic Data Creation

In [3]:
import pandas as pd
import random
import json

In [4]:
# Define a list of job titles and departments for variety
job_titles = [
    "Software Engineer", "Senior Software Engineer", "Data Scientist", "Product Manager",
    "Project Manager", "UX Designer", "QA Engineer", "DevOps Engineer", "CTO", "CEO"
]
departments = [
    "IT", "Engineering", "Data Science", "Product", "Project Management", "Design",
    "Quality Assurance", "Operations", "Executive"
]

# Define a list of office locations
office_locations = [
    "Chicago Office", "New York Office", "London Office", "Berlin Office", "Tokyo Office",
    "Sydney Office", "Toronto Office", "San Francisco Office", "Paris Office", "Singapore Office"
]

# Define a function to create a random employee entry
def create_employee(employee_id, first_name, last_name, job_title, department, manager_id=None):
    return {
        "employee_id": employee_id,
        "first_name": first_name,
        "last_name": last_name,
        "gender": random.choice(["Male", "Female"]),
        "date_of_birth": f"{random.randint(1950, 2000)}-{random.randint(1, 12):02}-{random.randint(1, 28):02}",
        "address": {
            "street": f"{random.randint(100, 999)} Main Street",
            "city": "Springfield",
            "state": "IL",
            "postal_code": "62704",
            "country": "USA"
        },
        "contact_details": {
            "email": f"{first_name.lower()}.{last_name.lower()}@example.com",
            "phone_number": f"+1-555-{random.randint(100, 999)}-{random.randint(1000, 9999)}"
        },
        "job_details": {
            "job_title": job_title,
            "department": department,
            "hire_date": f"{random.randint(2000, 2022)}-{random.randint(1, 12):02}-{random.randint(1, 28):02}",
            "employment_type": "Full-Time",
            "salary": random.randint(50000, 250000),
            "currency": "USD"
        },
        "work_location": {
            "nearest_office": random.choice(office_locations),
            "is_remote": random.choice([True, False])
        },
        "reporting_manager": manager_id,
        "skills": random.sample(
            ["JavaScript", "Python", "Node.js", "React", "Django", "Flask", "AWS", "Docker", "Kubernetes", "SQL"], 4
        ),
        "performance_reviews": [
            {
                "review_date": f"{random.randint(2020, 2023)}-{random.randint(1, 12):02}-{random.randint(1, 28):02}",
                "rating": round(random.uniform(3, 5), 1),
                "comments": random.choice([
                    "Exceeded expectations in the last project.",
                    "Consistently meets performance standards.",
                    "Needs improvement in time management.",
                    "Outstanding performance and dedication."
                ])
            },
            {
                "review_date": f"{random.randint(2019, 2022)}-{random.randint(1, 12):02}-{random.randint(1, 28):02}",
                "rating": round(random.uniform(3, 5), 1),
                "comments": random.choice([
                    "Exceeded expectations in the last project.",
                    "Consistently meets performance standards.",
                    "Needs improvement in time management.",
                    "Outstanding performance and dedication."
                ])
            }
        ],
        "benefits": {
            "health_insurance": random.choice(["Gold Plan", "Silver Plan", "Bronze Plan"]),
            "retirement_plan": "401K",
            "paid_time_off": random.randint(15, 30)
        },
        "emergency_contact": {
            "name": f"{random.choice(['Jane', 'Emily', 'Michael', 'Robert'])} {random.choice(['Doe', 'Smith', 'Johnson'])}",
            "relationship": random.choice(["Spouse", "Parent", "Sibling", "Friend"]),
            "phone_number": f"+1-555-{random.randint(100, 999)}-{random.randint(1000, 9999)}"
        },
        "notes": random.choice([
            "Promoted to Senior Software Engineer in 2020.",
            "Completed leadership training in 2021.",
            "Received Employee of the Month award in 2022.",
            "Actively involved in company hackathons and innovation challenges."
        ])
    }

# Generate 10 employee entries
employees = [
    create_employee("E123456", "John", "Doe", "Software Engineer", "IT", "M987654"),
    create_employee("E123457", "Jane", "Doe", "Senior Software Engineer", "IT", "M987654"),
    create_employee("E123458", "Emily", "Smith", "Data Scientist", "Data Science", "M987655"),
    create_employee("E123459", "Michael", "Brown", "Product Manager", "Product", "M987656"),
    create_employee("E123460", "Sarah", "Davis", "Project Manager", "Project Management", "M987657"),
    create_employee("E123461", "Robert", "Johnson", "UX Designer", "Design", "M987658"),
    create_employee("E123462", "David", "Wilson", "QA Engineer", "Quality Assurance", "M987659"),
    create_employee("E123463", "Chris", "Lee", "DevOps Engineer", "Operations", "M987660"),
    create_employee("E123464", "Sophia", "Garcia", "CTO", "Executive", None),
    create_employee("E123465", "Olivia", "Martinez", "CEO", "Executive", None)
]

In [5]:
# Convert to DataFrame
df_employees = pd.DataFrame(employees)

# Save DataFrame to CSV
csv_file_employees = 'synthetic_data_employees.csv'
df_employees.to_csv(csv_file_employees, index=False)

print(f"Synthetic employee data has been saved to {csv_file_employees}")

Synthetic employee data has been saved to synthetic_data_employees.csv


In [6]:
df_employees.head()

,employee_id,first_name,last_name,gender,date_of_birth,address,contact_details,job_details,work_location,reporting_manager,skills,performance_reviews,benefits,emergency_contact,notes
0,E123456,John,Doe,Male,1998-02-28,"{'street': '609 Main Street', 'city': 'Springf...","{'email': 'john.doe@example.com', 'phone_numbe...","{'job_title': 'Software Engineer', 'department...","{'nearest_office': 'Paris Office', 'is_remote'...",M987654,"[Node.js, Kubernetes, Flask, SQL]","[{'review_date': '2022-12-14', 'rating': 4.7, ...","{'health_insurance': 'Bronze Plan', 'retiremen...","{'name': 'Jane Doe', 'relationship': 'Sibling'...",Actively involved in company hackathons and in...
1,E123457,Jane,Doe,Female,1959-06-16,"{'street': '536 Main Street', 'city': 'Springf...","{'email': 'jane.doe@example.com', 'phone_numbe...","{'job_title': 'Senior Software Engineer', 'dep...","{'nearest_office': 'New York Office', 'is_remo...",M987654,"[Node.js, Django, JavaScript, Python]","[{'review_date': '2020-03-12', 'rating': 3.1, ...","{'health_insurance': 'Silver Plan', 'retiremen...","{'name': 'Robert Johnson', 'relationship': 'Fr...",Actively involved in company hackathons and in...
2,E123458,Emily,Smith,Male,1972-02-20,"{'street': '851 Main Street', 'city': 'Springf...","{'email': 'emily.smith@example.com', 'phone_nu...","{'job_title': 'Data Scientist', 'department': ...","{'nearest_office': 'Paris Office', 'is_remote'...",M987655,"[Flask, SQL, Kubernetes, React]","[{'review_date': '2021-07-04', 'rating': 4.7, ...","{'health_insurance': 'Gold Plan', 'retirement_...","{'name': 'Robert Smith', 'relationship': 'Pare...",Promoted to Senior Software Engineer in 2020.
3,E123459,Michael,Brown,Female,1995-01-01,"{'street': '999 Main Street', 'city': 'Springf...","{'email': 'michael.brown@example.com', 'phone_...","{'job_title': 'Product Manager', 'department':...","{'nearest_office': 'Tokyo Office', 'is_remote'...",M987656,"[JavaScript, SQL, Flask, Django]","[{'review_date': '2023-02-20', 'rating': 3.3, ...","{'health_insurance': 'Silver Plan', 'retiremen...","{'name': 'Jane Smith', 'relationship': 'Siblin...",Actively involved in company hackathons and in...
4,E123460,Sarah,Davis,Female,1992-02-21,"{'street': '168 Main Street', 'city': 'Springf...","{'email': 'sarah.davis@example.com', 'phone_nu...","{'job_title': 'Project Manager', 'department':...","{'nearest_office': 'Singapore Office', 'is_rem...",M987657,"[React, Python, Docker, SQL]","[{'review_date': '2023-03-18', 'rating': 4.1, ...","{'health_insurance': 'Silver Plan', 'retiremen...","{'name': 'Michael Smith', 'relationship': 'Sib...",Promoted to Senior Software Engineer in 2020.


## 1.2 Embedding Data For Vector Search

In [7]:
# Function to create a string representation of the employee's key attributes for embedding
def create_employee_string(employee):
    job_details = f"{employee['job_details']['job_title']} in {employee['job_details']['department']}"
    skills = ", ".join(employee['skills'])
    performance_reviews = " ".join([f"Rated {review['rating']} on {review['review_date']}: {review['comments']}" for review in employee['performance_reviews']])
    basic_info = f"{employee['first_name']} {employee['last_name']}, {employee['gender']}, born on {employee['date_of_birth']}"
    work_location = f"Works at {employee['work_location']['nearest_office']}, Remote: {employee['work_location']['is_remote']}"
    notes = employee['notes']

    return f"{basic_info}. Job: {job_details}. Skills: {skills}. Reviews: {performance_reviews}. Location: {work_location}. Notes: {notes}"

# Example usage with one employee
employee_string = create_employee_string(employees[0])
print(f"Here's what an employee string looks like: /n {employee_string}")

Here's what an employee string looks like: /n John Doe, Male, born on 1998-02-28. Job: Software Engineer in IT. Skills: Node.js, Kubernetes, Flask, SQL. Reviews: Rated 4.7 on 2022-12-14: Needs improvement in time management. Rated 4.6 on 2020-06-19: Needs improvement in time management.. Location: Works at Paris Office, Remote: False. Notes: Actively involved in company hackathons and innovation challenges.


In [8]:
# Apply the function to all employees
df_employees['employee_string'] = df_employees.apply(create_employee_string, axis=1)

In [9]:
# Generate an embedding using OpenAI's API
def get_embedding(text):
    """Generate an embedding for the given text using OpenAI's API."""

    # Check for valid input
    if not text or not isinstance(text, str):
        return None

    try:
        # Call OpenAI API to get the embedding
        embedding = openai.embeddings.create(
            input=text,
            model=OPEN_AI_EMBEDDING_MODEL, dimensions=OPEN_AI_EMBEDDING_MODEL_DIMENSION).data[0].embedding
        return embedding
    except Exception as e:
        print(f"Error in get_embedding: {e}")
        return None

# Apply the function to generate embeddings for all employees with error handling
try:
    df_employees['embedding'] = df_employees['employee_string'].apply(get_embedding)
    print("Embeddings generated for employees")
except Exception as e:
    print(f"Error applying embedding function to DataFrame: {e}")

Embeddings generated for employees


In [10]:
# Observe the new 'embedding' coloumn
df_employees.head()

,employee_id,first_name,last_name,gender,date_of_birth,address,contact_details,job_details,work_location,reporting_manager,skills,performance_reviews,benefits,emergency_contact,notes,employee_string,embedding
0,E123456,John,Doe,Male,1998-02-28,"{'street': '609 Main Street', 'city': 'Springf...","{'email': 'john.doe@example.com', 'phone_numbe...","{'job_title': 'Software Engineer', 'department...","{'nearest_office': 'Paris Office', 'is_remote'...",M987654,"[Node.js, Kubernetes, Flask, SQL]","[{'review_date': '2022-12-14', 'rating': 4.7, ...","{'health_insurance': 'Bronze Plan', 'retiremen...","{'name': 'Jane Doe', 'relationship': 'Sibling'...",Actively involved in company hackathons and in...,"John Doe, Male, born on 1998-02-28. Job: Softw...","[-0.03617718815803528, 0.027367066591978073, 0..."
1,E123457,Jane,Doe,Female,1959-06-16,"{'street': '536 Main Street', 'city': 'Springf...","{'email': 'jane.doe@example.com', 'phone_numbe...","{'job_title': 'Senior Software Engineer', 'dep...","{'nearest_office': 'New York Office', 'is_remo...",M987654,"[Node.js, Django, JavaScript, Python]","[{'review_date': '2020-03-12', 'rating': 3.1, ...","{'health_insurance': 'Silver Plan', 'retiremen...","{'name': 'Robert Johnson', 'relationship': 'Fr...",Actively involved in company hackathons and in...,"Jane Doe, Female, born on 1959-06-16. Job: Sen...","[-0.003852122463285923, 0.001388254459016025, ..."
2,E123458,Emily,Smith,Male,1972-02-20,"{'street': '851 Main Street', 'city': 'Springf...","{'email': 'emily.smith@example.com', 'phone_nu...","{'job_title': 'Data Scientist', 'department': ...","{'nearest_office': 'Paris Office', 'is_remote'...",M987655,"[Flask, SQL, Kubernetes, React]","[{'review_date': '2021-07-04', 'rating': 4.7, ...","{'health_insurance': 'Gold Plan', 'retirement_...","{'name': 'Robert Smith', 'relationship': 'Pare...",Promoted to Senior Software Engineer in 2020.,"Emily Smith, Male, born on 1972-02-20. Job: Da...","[-0.011636233888566494, 0.01979971118271351, 0..."
3,E123459,Michael,Brown,Female,1995-01-01,"{'street': '999 Main Street', 'city': 'Springf...","{'email': 'michael.brown@example.com', 'phone_...","{'job_title': 'Product Manager', 'department':...","{'nearest_office': 'Tokyo Office', 'is_remote'...",M987656,"[JavaScript, SQL, Flask, Django]","[{'review_date': '2023-02-20', 'rating': 3.3, ...","{'health_insurance': 'Silver Plan', 'retiremen...","{'name': 'Jane Smith', 'relationship': 'Siblin...",Actively involved in company hackathons and in...,"Michael Brown, Female, born on 1995-01-01. Job...","[-0.003844072110950947, -0.0012454793322831392..."
4,E123460,Sarah,Davis,Female,1992-02-21,"{'street': '168 Main Street', 'city': 'Springf...","{'email': 'sarah.davis@example.com', 'phone_nu...","{'job_title': 'Project Manager', 'department':...","{'nearest_office': 'Singapore Office', 'is_rem...",M987657,"[React, Python, Docker, SQL]","[{'review_date': '2023-03-18', 'rating': 4.1, ...","{'health_insurance': 'Silver Plan', 'retiremen...","{'name': 'Michael Smith', 'relationship': 'Sib...",Promoted to Senior Software Engineer in 2020.,"Sarah Davis, Female, born on 1992-02-21. Job: ...","[0.018026115372776985, 0.00723855709657073, 0...."


## 1.3 Data Ingestion into MongoDB Database

**Steps to creating a MongoDB Database**
- [Register for a free MongoDB Atlas Account](https://www.mongodb.com/cloud/atlas/register?utm_campaign=devrel&utm_source=workshop&utm_medium=organic_social&utm_content=rag%20to%20agents%20notebook&utm_term=richmond.alake)
- [Create a Cluster](https://www.mongodb.com/docs/guides/atlas/cluster/)
- [Get your connection string](https://www.mongodb.com/docs/guides/atlas/connection-string/)





In [11]:
os.environ["MONGO_URI"] = <place MONGO Atlas URI here>

MONGO_URI = os.environ.get("MONGO_URI")

In [12]:
from pymongo.mongo_client import MongoClient

In [13]:
DATABASE_NAME = "demo_company_employees"
COLLECTION_NAME = "employees_records"

In [14]:
def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""

    # gateway to interacting with a MongoDB database cluster
    client = MongoClient(mongo_uri, appname="devrel.showcase.workshop.rag_to_agent")
    print("Connection to MongoDB successful")
    return client

In [15]:
if not MONGO_URI:
    print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(MONGO_URI)

# Pymongo client of database and collection
db = mongo_client.get_database(DATABASE_NAME)
collection = db.get_collection(COLLECTION_NAME)

Connection to MongoDB successful


In [16]:
documents = df_employees.to_dict('records')

In [17]:
# Clean up collection of exisiting record
collection.delete_many({})

DeleteResult({'n': 0, 'electionId': ObjectId('7fffffff000000000000044d'), 'opTime': {'ts': Timestamp(1725223492, 41), 't': 1101}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1725223492, 41), 'signature': {'hash': b'\xb9\xbf-\xb5\xe3\xe5\xcf\x9dMAa\xdd\x99\xa2\x94\x05\x8c\xe3\xf2\x0f', 'keyId': 7366517091924443137}}, 'operationTime': Timestamp(1725223492, 41)}, acknowledged=True)

## 1.4 Vector Index Creation

- [Create an Atlas Vector Search Index](https://www.mongodb.com/docs/compass/current/indexes/create-vector-search-index/)

- If you are following this notebook ensure that you are creating a vector search index for the right database(demo_company_employees) and collection(employees_records)

Below is the vector search index definition for this notebook

```json
{
  "fields": [
    {
      "numDimensions": 1536,
      "path": "embedding",
      "similarity": "cosine",
      "type": "vector"
    }
  ]
}
```

- Give your vector search index the name "vector_index" if you are following this notebook


In [18]:
# Ingest data into MongoDB Database
collection.insert_many(documents)
print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


## 1.5 RAG with MongoDB

In [19]:
def vector_search(user_query, collection, vector_index="vector_index"):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    db (MongoClient.database): The database object.
    collection (MongoCollection): The MongoDB collection to search.
    additional_stages (list): Additional aggregation stages to include in the pipeline.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search stage
    vector_search_stage = {
        "$vectorSearch": {
            "index": vector_index, # specifies the index to use for the search
            "queryVector": query_embedding, # the vector representing the query
            "path": "embedding", # field in the documents containing the vectors to search against
            "numCandidates": 150, # number of candidate matches to consider
            "limit": 5 # return top 20 matches
        }
    }

    # Define the aggregate pipeline with the vector search stage and additional stages
    pipeline = [vector_search_stage]

    # Execute the search
    results = collection.aggregate(pipeline)

    return list(results)

## 1.6 Handling User Query

In [20]:
def handle_user_query(query, collection):

  get_knowledge = vector_search(query, collection)

  # Concatenate the search results to reflect the employee profile
  search_result = ''

  for result in get_knowledge:
      reporting_manager = result.get('reporting_manager')
      if isinstance(reporting_manager, dict):
          manager_id = reporting_manager.get('manager_id', 'N/A')
      else:
          manager_id = 'N/A'

      employee_profile = f"""
      Employee ID: {result.get('employee_id', 'N/A')}
      Name: {result.get('first_name', 'N/A')} {result.get('last_name', 'N/A')}
      Gender: {result.get('gender', 'N/A')}
      Date of Birth: {result.get('date_of_birth', 'N/A')}
      Address: {result.get('address', {}).get('street', 'N/A')}, {result.get('address', {}).get('city', 'N/A')}, {result.get('address', {}).get('state', 'N/A')}, {result.get('address', {}).get('postal_code', 'N/A')}, {result.get('address', {}).get('country', 'N/A')}
      Contact Details: Email - {result.get('contact_details', {}).get('email', 'N/A')}, Phone - {result.get('contact_details', {}).get('phone_number', 'N/A')}
      Job Details: Title - {result.get('job_details', {}).get('job_title', 'N/A')}, Department - {result.get('job_details', {}).get('department', 'N/A')}, Hire Date - {result.get('job_details', {}).get('hire_date', 'N/A')}, Type - {result.get('job_details', {}).get('employment_type', 'N/A')}, Salary - {result.get('job_details', {}).get('salary', 'N/A')} {result.get('job_details', {}).get('currency', 'N/A')}
      Work Location: Nearest Office - {result.get('work_location', {}).get('nearest_office', 'N/A')}, Remote - {result.get('work_location', {}).get('is_remote', 'N/A')}
      Reporting Manager: ID - {manager_id}
      Skills: {', '.join(result.get('skills', ['N/A']))}
      Performance Reviews: {', '.join([f"Date: {review.get('review_date', 'N/A')}, Rating: {review.get('rating', 'N/A')}, Comments: {review.get('comments', 'N/A')}" for review in result.get('performance_reviews', [])])}
      Benefits: Health Insurance - {result.get('benefits', {}).get('health_insurance', 'N/A')}, Retirement Plan - {result.get('benefits', {}).get('retirement_plan', 'N/A')}, PTO - {result.get('benefits', {}).get('paid_time_off', 'N/A')} days
      Emergency Contact: Name - {result.get('emergency_contact', {}).get('name', 'N/A')}, Relationship - {result.get('emergency_contact', {}).get('relationship', 'N/A')}, Phone - {result.get('emergency_contact', {}).get('phone_number', 'N/A')}
      Notes: {result.get('notes', 'N/A')}
      """
      search_result += employee_profile + "\n"

  prompt = "Answer this user query: " + query + " with the following context: " + search_result
  print("Uncompressed Prompt:\n")
  print(prompt)

  completion = openai.chat.completions.create(
      model=OPEN_AI_MODEL,
      messages=[
          {"role": "system", "content": "You are an Human Resource System within a corporate company."},
          {"role": "user", "content": prompt}
      ]
  )

  return (completion.choices[0].message.content), search_result

In [21]:
# Conduct query with retrival of sources
query = "Who is the CEO?"
response, source_information = handle_user_query(query, collection)

print(f"Response: {response}")

Uncompressed Prompt:

Answer this user query: Who is the CEO? with the following context: 
      Employee ID: E123465
      Name: Olivia Martinez
      Gender: Female
      Date of Birth: 1973-04-08
      Address: 741 Main Street, Springfield, IL, 62704, USA
      Contact Details: Email - olivia.martinez@example.com, Phone - +1-555-693-5504
      Job Details: Title - CEO, Department - Executive, Hire Date - 2009-11-16, Type - Full-Time, Salary - 206881 USD
      Work Location: Nearest Office - San Francisco Office, Remote - True
      Reporting Manager: ID - N/A
      Skills: Python, React, JavaScript, AWS
      Performance Reviews: Date: 2021-08-14, Rating: 3.2, Comments: Consistently meets performance standards., Date: 2019-11-04, Rating: 3.7, Comments: Outstanding performance and dedication.
      Benefits: Health Insurance - Gold Plan, Retirement Plan - 401K, PTO - 29 days
      Emergency Contact: Name - Jane Doe, Relationship - Friend, Phone - +1-555-972-3144
      Notes: Actively

## 1.7 Handling User Query (With Prompt Compression)

In [22]:
# Uncomment and run the following line if a hardware accelerator(gpu) is available in your development environment:
# ! pip install optimum auto-gptq

In [23]:
from llmlingua import PromptCompressor

llm_lingua = PromptCompressor(
    model_name="microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank",
    model_config={"revision": "main"},
    use_llmlingua2=True,
    device_map="cpu" # change to 'cuda' if gpu is availabe on device
)

# Function definition
def compress_query_prompt(context):

  compressed_prompt = llm_lingua.compress_prompt(
    str(context),
    rate=0.33,
    force_tokens=["!", ".", "?", "\n"],
    drop_consecutive=True,
  )

  print("------")
  print(compressed_prompt)
  print("-------")

  return compressed_prompt

config.json:   0%|          | 0.00/875 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

In [24]:
import pprint

def handle_user_query_with_compression(query, collection):

  get_knowledge = vector_search(query, collection)

  # Concatenate the search results to reflect the employee profile
  search_result = ''

  for result in get_knowledge:
      employee_profile = f"""
      Employee ID: {result.get('employee_id', 'N/A')}
      Name: {result.get('first_name', 'N/A')} {result.get('last_name', 'N/A')}
      Gender: {result.get('gender', 'N/A')}
      Date of Birth: {result.get('date_of_birth', 'N/A')}
      Address: {result.get('address', {}).get('street', 'N/A')}, {result.get('address', {}).get('city', 'N/A')}, {result.get('address', {}).get('state', 'N/A')}, {result.get('address', {}).get('postal_code', 'N/A')}, {result.get('address', {}).get('country', 'N/A')}
      Contact Details: Email - {result.get('contact_details', {}).get('email', 'N/A')}, Phone - {result.get('contact_details', {}).get('phone_number', 'N/A')}
      Job Details: Title - {result.get('job_details', {}).get('job_title', 'N/A')}, Department - {result.get('job_details', {}).get('department', 'N/A')}, Hire Date - {result.get('job_details', {}).get('hire_date', 'N/A')}, Type - {result.get('job_details', {}).get('employment_type', 'N/A')}, Salary - {result.get('job_details', {}).get('salary', 'N/A')} {result.get('job_details', {}).get('currency', 'N/A')}
      Work Location: Nearest Office - {result.get('work_location', {}).get('nearest_office', 'N/A')}, Remote - {result.get('work_location', {}).get('is_remote', 'N/A')}
      Skills: {', '.join(result.get('skills', ['N/A']))}
      Performance Reviews: {', '.join([f"Date: {review.get('review_date', 'N/A')}, Rating: {review.get('rating', 'N/A')}, Comments: {review.get('comments', 'N/A')}" for review in result.get('performance_reviews', [])])}
      Benefits: Health Insurance - {result.get('benefits', {}).get('health_insurance', 'N/A')}, Retirement Plan - {result.get('benefits', {}).get('retirement_plan', 'N/A')}, PTO - {result.get('benefits', {}).get('paid_time_off', 'N/A')} days
      Emergency Contact: Name - {result.get('emergency_contact', {}).get('name', 'N/A')}, Relationship - {result.get('emergency_contact', {}).get('relationship', 'N/A')}, Phone - {result.get('emergency_contact', {}).get('phone_number', 'N/A')}
      Notes: {result.get('notes', 'N/A')}
      """
      search_result += employee_profile + "\n"

  # Prepare information for compression
  query_info = {
    'demonstration_str': search_result,  # Results from information retrieval process
    'instruction': "Write a high-quality answer for the given question using only the provided search results.",
    'question': query
  }

  # Compress the query prompt
  compressed_prompt = compress_query_prompt(query_info)

  prompt =  f"Answer this user query: {query} with the following context:\n{compressed_prompt}"
  print("Compressed Prompt:\n")
  pprint.pprint(prompt)

  completion = openai.chat.completions.create(
      model=OPEN_AI_MODEL,
      messages=[
          {"role": "system", "content": "You are an Human Resource System within a corporate company."},
          {"role": "user", "content": prompt}
      ]
  )

  return (completion.choices[0].message.content), search_result

In [25]:
from llmlingua import PromptCompressor

question = "Who is the CEO?"
instruction="Write a high-quality answer for the given question using only the provided search results"
prompt=collection

llm_lingua = PromptCompressor("microsoft/phi-2")

#llm_lingua = PromptCompressor(
#    model_name="microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank",
#    model_config={"revision": "main"},
#    use_llmlingua2=True,
#    device_map="cpu" # change to 'cuda' if gpu is availabe on device
#)

compressed_prompt = llm_lingua.compress_prompt(prompt, instruction, question, target_token=200,rate=0.33,
    force_tokens=["!", ".", "?", "\n"],
    drop_consecutive=True,)

# > {'compressed_prompt': 'Question: Sam bought a dozen boxes, each with 30 highlighter pens inside, for $10 each box. He reanged five of boxes into packages of sixlters each and sold them $3 per. He sold the rest theters separately at the of three pens $2. How much did make in total, dollars?\nLets think step step\nSam bought 1 boxes x00 oflters.\nHe bought 12 * 300ters in total\nSam then took 5 boxes 6ters0ters.\nHe sold these boxes for 5 *5\nAfterelling these  boxes there were 3030 highlighters remaining.\nThese form 330 / 3 = 110 groups of three pens.\nHe sold each of these groups for $2 each, so made 110 * 2 = $220 from them.\nIn total, then, he earned $220 + $15 = $235.\nSince his original cost was $120, he earned $235 - $120 = $115 in profit.\nThe answer is 115',
#  'origin_tokens': 2365,
#  'compressed_tokens': 211,
#  'ratio': '11.2x',
#  'saving': ', Saving $0.1 in GPT-4.'}

## Or use the phi-2 model,
#llm_lingua = PromptCompressor("microsoft/phi-2")

## Or use the quantation model, like TheBloke/Llama-2-7b-Chat-GPTQ, only need <8GB GPU memory.
## Before that, you need to pip install optimum auto-gptq
#llm_lingua = PromptCompressor("TheBloke/Llama-2-7b-Chat-GPTQ", model_config={"revision": "main"})

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

AssertionError: Torch not compiled with CUDA enabled

In [29]:
# Conduct query with retrival of sources
query = "Who is the CEO?"
response, source_information = handle_user_query_with_compression(query, collection)

print(f"Response: {response}")

Token indices sequence length is longer than the specified maximum sequence length for this model (1423 > 512). Running this sequence through the model will result in indexing errors


------
{'compressed_prompt': 'Employee ID E123465 Olivia Martinez Female Birth 1992 - 04 - 03 728 Main Street Springfield IL 62704 olivia. martinez. 555 644 1042 09 - 16 238242 USD JavaScript Python Performance 2021 - 12 - 14 4. 5. 2019 08 - 28. 9 performance dedication. Benefits Health Insurance Bronze Plan Retirement Plan 401K PTO 27 Emergency Jane Doe 555 - 526 - 5724. Employee ID E123464 Sophia Garcia Male 1955 - 08 - 16 686 Main Street Springfield 62704. garcia. 478 7592 CTO 07 27 168553 USD. 2020 - 11 - 01 4. 8. 2019 - 07 - 13 3. 1 performance standards.Benefits Health Insurance - Gold Plan Retirement Plan 401K 25 days Emergency Contact Michael Smith Relationship Sibling - 555 - 762 - 2261 Senior Software Engineer 2020. Employee ID E123463 Chris Lee Birth 1992 - 06 - 26 590 Main Street Springfield IL 62704. - 521 - 6570 DevOps Engineer 2017 12 - 05 86282 USD Office New York 2023 - 08 - 16. time management. Plan 401K 17 days Emily Doe Spouse 555 - 694 - 7984 innovation challenges.

# Part 2: RAG Application: HR Use Case (POLM AI Stack)


### RAG with Langchain and MongoDB

In [ ]:
!pip install --upgrade --quiet langchain langchain-mongodb langchain-openai langchain_community pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 14.5 MB/s eta 0:00:00


In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_mongodb import MongoDBAtlasVectorSearch

embedding_model = OpenAIEmbeddings(model=OPEN_AI_EMBEDDING_MODEL, dimensions=OPEN_AI_EMBEDDING_MODEL_DIMENSION)

# Vector Store Creation
vector_store = MongoDBAtlasVectorSearch.from_connection_string(
    connection_string=MONGO_URI,
    namespace=DATABASE_NAME + "." + COLLECTION_NAME,
    embedding= embedding_model,
    index_name="vector_index",
    text_key="employee_string"
)

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
from langchain.prompts import PromptTemplate

# Define a prompt template
template = """
Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
Question: {question}
"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [ ]:
llm = ChatOpenAI()

In [ ]:
def format_docs(docs):
   return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Construct a chain to answer questions on your data
rag_chain = (
   { "context": retriever | format_docs, "question": RunnablePassthrough()}
   | custom_rag_prompt
   | llm
   | StrOutputParser()
)
# Prompt the chain
question = "Who is the CEO??"
answer = rag_chain.invoke(question)

In [ ]:
print("Question: " + question)
print("Answer: " + answer)

Question: Who is the CEO??
Answer: Olivia Martinez is the CEO.


#### Prompt Compression with LangChain and LLMLingua

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import LLMLinguaCompressor

In [ ]:
compressor = LLMLinguaCompressor(model_name="openai-community/gpt2", device_map="cpu")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
compressed_docs = compression_retriever.invoke(
    "Who is the CEO?"
)
print(compressed_docs)

[Document(page_content='Olivia Martinez, Female, born on 1952-01-05. Job: CEO in Executive. Skills: Docker, React, Flask, Node.js. Reviews: Rated 4.5 on 2020-10-09: Needs improvement in time management. Rated 3.2 on 2021-01-08: Consistently meets performance standards.. Location: Works at Sydney Office, Remote: False. Notes: Received Employee of the Month award in 2022.', metadata={'_id': {'$oid': '6669c346ce0888213014cce4'}, 'employee_id': 'E123465', 'first_name': 'Olivia', 'last_name': 'Martinez', 'gender': 'Female', 'date_of_birth': '1952-01-05', 'address': {'street': '959 Main Street', 'city': 'Springfield', 'state': 'IL', 'postal_code': '62704', 'country': 'USA'}, 'contact_details': {'email': 'olivia.martinez@example.com', 'phone_number': '+1-555-675-3033'}, 'job_details': {'job_title': 'CEO', 'department': 'Executive', 'hire_date': '2015-09-16', 'employment_type': 'Full-Time', 'salary': 53606, 'currency': 'USD'}, 'work_location': {'nearest_office': 'Sydney Office', 'is_remote': F

In [ ]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm, retriever=compression_retriever)

In [ ]:
chain.invoke({"query": "Who is the CEO?"})

{'query': 'Who is the CEO?', 'result': 'Olivia Martinez is the CEO.'}

### RAG with LlamaIndex and MongoDB (Coming Soon)

### RAG with HayStack and MongoDB (Coming Soon)

# Part 3: AI Agent Application: HR Use Case (POLM AI Stack)


### AI Agents with langChain and MongoDB (Coming Soon)

### AI Agents with LlamaIndex and MongoDB (Coming Soon)

### AI Agents with HayStack and MongoDB (Coming Soon)